In [7]:
import sqlite3

old_db_conn = sqlite3.connect('./backup/old.db')
new_db_conn = sqlite3.connect('./kwu-lecture-recommendation-v2-updating.db')

old_cursor = old_db_conn.cursor()
new_cursor = new_db_conn.cursor()


old_cursor.execute('''
SELECT year, semester, lecNumber, lecIsPNP, lecIsEngeneering, lecRequirementClass, 
       lecIsTBL, lecIsPBL, lecIsSeminar, lecIsSmall, lecIsConvergence, 
       lecIsTeamTeach, lecIsFocus, lecIsArt, lecOverview, lecEvaluateRatio
FROM LectureTable
''')

rows = old_cursor.fetchall()

for row in rows:
    year = row[0]
    semester = row[1]
    lecNumber = row[2]
    lecIsPNP = row[3]
    lecIsEngineering = row[4]
    lecRequirementClass = row[5]
    lecIsTBL = row[6]
    lecIsPBL = row[7]
    lecIsSeminar = row[8]
    lecIsSmall = row[9]
    lecIsConvergence = row[10]
    lecIsTeamTeach = row[11]
    lecIsFocus = row[12]
    lecIsArt = row[13]
    lecOverview = row[14]
    lecEvaluateRatio = row[15]

    new_cursor.execute('''
    SELECT lectureID FROM LectureList
    WHERE year = ? AND semester = ? AND lecNumber = ?
    ''', (year, semester, lecNumber))

    lecture_id_row = new_cursor.fetchone()

    if lecture_id_row:
        lectureID = lecture_id_row[0]

        new_cursor.execute('''
        INSERT INTO LectureDetailData (lectureID, isPNP, isEngineering, isTBL, isPBL, 
                                       isSeminar, isSmall, isConvergence, 
                                       isTeamTeaching, isFocus, isArt, 
                                       Overview, evaluationRatio)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (lectureID, lecIsPNP, lecIsEngineering, lecIsTBL, lecIsPBL,
              lecIsSeminar, lecIsSmall, lecIsConvergence,
              lecIsTeamTeach, lecIsFocus, lecIsArt,
              lecOverview, lecEvaluateRatio))


new_db_conn.commit()


old_db_conn.close()
new_db_conn.close()

print("done")

done


In [5]:
# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')#
cursor = conn.cursor()


q = """CREATE VIEW LectureDetailView AS
SELECT
    l.lectureID,
    l.year,
    l.semester,
    l.lecNumber,
    l.lecName,
    l.lecProfessor,
    l.lecClassification,
    l.lecTheme,
    l.lecCredit,
    l.lecTime,
    l.lecWeekTime,
    l.lecClassroom,
    l.isLecClose,
    ld.takenPeople1yearsAgo,
    ld.takenPeople2yearsAgo,
    ld.takenPeople3yearsAgo,
    ld.ForeignLanguage,
    ld.percentageOfOnline,
    ld.isPNP,
    ld.isEngineering,
    ld.isTBL,
    ld.isPBL,
    ld.isSeminar,
    ld.isSmall,
    ld.isConvergence,
    ld.isTeamTeaching,
    ld.isFocus,
    ld.isExperimentDesign,
    ld.isELearning,
    ld.isArt,
    ld.representCompetency,
    ld.learningGoalNmethod,
    ld.Overview,
    ld.evaluationRatio,
    ld.mainBook,
    ld.scheduleNcontent,
    le.everytimeURL,
    le.star,
    le.assignmentAmount,
    le.teamPlayAmount,
    le.gradeAmount,
    le.reviewSummary,
    le.checkAttend,
    le.testNum
FROM
    LectureList l
LEFT JOIN
    LectureDetailData ld ON l.lectureID = ld.lectureID
LEFT JOIN
    LectureEverytimeData le ON l.lectureID = le.lectureID;"""



conn.execute(q)
conn.commit()  # 변경사항을 저장합니다.
conn.close()   # 연결을 닫습니다.

OperationalError: view LectureDetailView already exists

In [3]:
import csv
import sqlite3


new_db_conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
new_cursor = new_db_conn.cursor()

def convert_to_yn(value):
    return 1 if value.lower() == 'checked' else 0

# CSV 파일 열 이름과 데이터베이스 필드 매핑
field_mapping = {
    '년도학기': 'year',
    '학정번호': 'lecNumber',
    '강의소개': 'Overview',
    '영어강의 비율': 'evaluationRatio',
    'TBL강의': 'isTBL',
    'PBL강의': 'isPBL',
    '세미나강의': 'isSeminar',
    '소규모강의': 'isSmall',
    '융합강의': 'isConvergence',
    '팀티칭강의': 'isTeamTeaching',
    '집중이수제강의': 'isFocus',
    '예체능': 'isArt'
}

# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v5.db')
cursor = conn.cursor()

# CSV 파일 읽기
with open('lecture_plan_24_2(1).csv', 'r', encoding='cp949') as file:
    csv_reader = csv.DictReader(file)
    
    for row in csv_reader:
        # 학정번호가 있는 경우에만 처리
        if row['학정번호']:
            # 년도와 학기 분리
            year_semester = row['년도학기'].split('-')
            # 고정 값
            year = 24
            semester = '2학기'
            # LectureList 테이블에서 lectureID 조회
            new_cursor.execute('''
            SELECT lectureID FROM LectureList
            WHERE year = ? AND semester = ? AND lecNumber = ?
            ''', (year, semester, row['학정번호']))

            lecture_id_row = cursor.fetchone()

            if lecture_id_row:
                lectureID = lecture_id_row[0]

                # LectureDetailData 테이블에 삽입할 데이터 준비
                insert_data = {
                    'lectureID': lectureID,
                    'Overview': row.get('강의소개', ''),
                    'evaluationRatio': row.get('영어강의 비율', ''),
                    'isTBL': convert_to_yn(row.get('TBL강의', '')),
                    'isPBL': convert_to_yn(row.get('PBL강의', '')),
                    'isSeminar': convert_to_yn(row.get('세미나강의', '')),
                    'isSmall': convert_to_yn(row.get('소규모강의', '')),
                    'isConvergence': convert_to_yn(row.get('융합강의', '')),
                    'isTeamTeaching': convert_to_yn(row.get('팀티칭강의', '')),
                    'isFocus': convert_to_yn(row.get('집중이수제강의', '')),
                    'isArt': convert_to_yn(row.get('예체능', ''))
                }

                # SQL 쿼리 생성
                columns = ', '.join(insert_data.keys())
                placeholders = ', '.join(['?' for _ in insert_data])
                sql = f'''
                INSERT INTO LectureDetailData ({columns})
                VALUES ({placeholders})
                '''

                # 데이터 삽입 실행
                cursor.execute(sql, list(insert_data.values()))

# 변경사항 저장 및 연결 종료
conn.commit()
conn.close()

print("데이터 가져오기가 완료되었습니다.")

데이터 가져오기가 완료되었습니다.


In [1]:
import csv
import sqlite3

def convert_to_yn(value):
    return 1 if value.lower() == 'checked' else 0

# CSV 파일 열 이름과 데이터베이스 필드 매핑
field_mapping = {
    '년도학기': 'year',
    '학정번호': 'lecNumber',
    '강의소개': 'Overview',
    '영어강의 비율': 'evaluationRatio',
    'TBL강의': 'isTBL',
    'PBL강의': 'isPBL',
    '세미나강의': 'isSeminar',
    '소규모강의': 'isSmall',
    '융합강의': 'isConvergence',
    '팀티칭강의': 'isTeamTeaching',
    '집중이수제강의': 'isFocus',
    '예체능': 'isArt',
    '원격 비율': 'percentageOfOnline'
}

try:
    # 데이터베이스 연결
    conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
    cursor = conn.cursor()

    # CSV 파일 읽기
    with open('lecture_plan_24_2(1).csv', 'r', encoding='cp949') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            # 학정번호가 있는 경우에만 처리
            if row['학정번호']:
                # 년도와 학기 분리
                year_semester = row['년도학기'].split('-')
                # 고정 값
                year = 24
                semester = '2학기'

                # LectureList 테이블에서 lectureID 조회
                cursor.execute('''
                SELECT lectureID FROM LectureList
                WHERE year = ? AND semester = ? AND lecNumber = ?
                ''', (year, semester, row['학정번호']))
                
                lecture_id_row = cursor.fetchone()
                
                if lecture_id_row:
                    lectureID = lecture_id_row[0]
                    
                    # LectureDetailData 테이블에 삽입할 데이터 준비
                    insert_data = {
                        'lectureID': lectureID,
                        'Overview': row.get('교과목 개요', ''),
                        'evaluationRatio': row.get('강의 비율', ''),
                        'isTBL': convert_to_yn(row.get('TBL강의', '')),
                        'isPBL': convert_to_yn(row.get('PBL강의', '')),
                        'isSeminar': convert_to_yn(row.get('세미나강의', '')),
                        'isSmall': convert_to_yn(row.get('소규모강의', '')),
                        'isConvergence': convert_to_yn(row.get('융합강의', '')),
                        'isTeamTeaching': convert_to_yn(row.get('팀티칭강의', '')),
                        'isFocus': convert_to_yn(row.get('집중이수제강의', '')),
                        'isArt': convert_to_yn(row.get('예체능', '')),
                        'percentageOfOnline': row.get('원격 비율', '')
                    }
                    
                    # SQL 쿼리 생성
                    columns = ', '.join(insert_data.keys())
                    placeholders = ', '.join(['?' for _ in insert_data])
                    sql = f'''
                    INSERT INTO LectureDetailData ({columns})
                    VALUES ({placeholders})
                    '''
                    
                    # 데이터 삽입 실행
                    cursor.execute(sql, list(insert_data.values()))
                    print(f"Inserted data for lecture: {row['학정번호']}")
                else:
                    print(f"No matching lectureID found for: {row['학정번호']}")

    # 변경사항 저장
    conn.commit()
    print("데이터 가져오기가 완료되었습니다.")

except sqlite3.Error as e:
    print(f"데이터베이스 오류: {e}")
except csv.Error as e:
    print(f"CSV 파일 오류: {e}")
except Exception as e:
    print(f"예상치 못한 오류: {e}")
finally:
    # 연결 종료
    if conn:
        conn.close()

Inserted data for lecture: 0000-1-8128-01
Inserted data for lecture: 0000-1-8128-02
Inserted data for lecture: I000-1-3362-01
Inserted data for lecture: I000-1-3362-02
Inserted data for lecture: I000-1-3362-03
Inserted data for lecture: I000-1-3362-04
Inserted data for lecture: I000-1-3362-05
Inserted data for lecture: I000-1-3362-06
Inserted data for lecture: I000-1-3362-07
Inserted data for lecture: I000-1-3362-08
Inserted data for lecture: I000-1-3362-09
Inserted data for lecture: I000-1-3362-10
Inserted data for lecture: I000-1-3362-11
Inserted data for lecture: I000-1-3362-12
Inserted data for lecture: I000-1-3362-13
Inserted data for lecture: I000-1-3362-14
Inserted data for lecture: 1000-1-3362-01
Inserted data for lecture: 1000-1-3362-02
Inserted data for lecture: 1000-1-3362-03
Inserted data for lecture: 1000-1-3362-04
Inserted data for lecture: 1000-1-3362-05
Inserted data for lecture: 1000-1-3362-06
Inserted data for lecture: 3000-1-3362-01
Inserted data for lecture: 3000-1-

Inserted data for lecture: 0000-2-4970-08
Inserted data for lecture: 0000-2-4970-09
Inserted data for lecture: 0000-2-4970-10
Inserted data for lecture: 0000-2-4970-11
Inserted data for lecture: 0000-2-4970-12
Inserted data for lecture: 0000-2-4969-01
Inserted data for lecture: 0000-2-6020-01
Inserted data for lecture: 0000-2-7724-01
Inserted data for lecture: 0000-2-0819-01
Inserted data for lecture: 0000-2-0819-02
Inserted data for lecture: 0000-2-0819-03
Inserted data for lecture: 0000-2-0819-04
Inserted data for lecture: 0000-2-0819-05
Inserted data for lecture: 0000-3-1942-01
Inserted data for lecture: 0000-3-8120-01
Inserted data for lecture: 0000-1-3416-01
Inserted data for lecture: 0000-1-3416-02
Inserted data for lecture: 0000-1-3416-03
Inserted data for lecture: 0000-1-3416-04
Inserted data for lecture: 0000-1-3416-05
Inserted data for lecture: 0000-1-3414-01
Inserted data for lecture: 0000-1-9884-01
Inserted data for lecture: 0000-1-9884-02
Inserted data for lecture: 0000-1-

In [4]:
import csv
import sqlite3

new_db_conn = sqlite3.connect('kwu-lecture-recommendation-v4 copy.db')

new_cursor = new_db_conn.cursor()

# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
cursor = conn.cursor()

# isLecClose 값을 변환하는 함수
def convert_is_lec_close(value):
    return '1' if value.upper() == 'TRUE' else '0'

# CSV 파일 읽기 및 데이터베이스에 삽입
with open('yn_final_kwu-timetable5.csv', 'r', encoding='cp949') as csvfile:
    csvreader = csv.DictReader(csvfile)
    
    for row in csvreader:
        # 데이터 추출 및 변환
        lecNumber = row['학정번호']
        lecName = row['과목명']
        lecProfessor = row['담당교수']
        lecClassification = row['이수']
        lecTheme = row['테마']
        lecCredit = row['학점']
        lecTime = row['강의시간']
        lecWeekTime = row['시수']
        lecClassroom = row['강의실']
        isLecClose = convert_is_lec_close(row['폐강'])

        # 고정 값
        year = '24'
        semester = '2학기'
        new_cursor.execute('''
        SELECT lectureID FROM LectureList
        WHERE year = ? AND semester = ? AND lecNumber = ?
        ''', (year, semester, lecNumber))

        lecture_id_row = new_cursor.fetchone()

        if lecture_id_row:
            lectureID = lecture_id_row[0]
            # 데이터베이스에 삽입 또는 업데이트
            cursor.execute('''
            INSERT OR REPLACE INTO LectureList 
            (lecNumber, lecName, lecProfessor, lecClassification, lecTheme, 
            lecCredit, lecTime, lecWeekTime, lecClassroom, isLecClose, 
            year, semester)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (lecNumber, lecName, lecProfessor, lecClassification, lecTheme,
                lecCredit, lecTime, lecWeekTime, lecClassroom, isLecClose,
                year, semester))

# 변경사항 커밋 및 연결 종료
conn.commit()
conn.close()

print("Data insertion completed.")

KeyError: '과목명'

In [ ]:
import csv
import sqlite3


new_db_conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
new_cursor = new_db_conn.cursor()

def convert_to_yn(value):
    return 1 if value.lower() == 'checked' else 0

# CSV 파일 열 이름과 데이터베이스 필드 매핑
field_mapping = {
    '년도학기': 'year',
    '학정번호': 'lecNumber',
    '강의소개': 'Overview',
    '영어강의 비율': 'evaluationRatio',
    'TBL강의': 'isTBL',
    'PBL강의': 'isPBL',
    '세미나강의': 'isSeminar',
    '소규모강의': 'isSmall',
    '융합강의': 'isConvergence',
    '팀티칭강의': 'isTeamTeaching',
    '집중이수제강의': 'isFocus',
    '예체능': 'isArt'
}

# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v5.db')
cursor = conn.cursor()

# CSV 파일 읽기
with open('lecture_plan_24_2(1).csv', 'r', encoding='cp949') as file:
    csv_reader = csv.DictReader(file)
    
    for row in csv_reader:
        # 학정번호가 있는 경우에만 처리
        if row['학정번호']:
            # 년도와 학기 분리
            year_semester = row['년도학기'].split('-')
            # 고정 값
            year = 24
            semester = '2학기'
            # LectureList 테이블에서 lectureID 조회
            new_cursor.execute('''
            SELECT lectureID FROM LectureList
            WHERE year = ? AND semester = ? AND lecNumber = ?
            ''', (year, semester, row['학정번호']))

            lecture_id_row = cursor.fetchone()

            if lecture_id_row:
                lectureID = lecture_id_row[0]

                # LectureDetailData 테이블에 삽입할 데이터 준비
                insert_data = {
                    'lectureID': lectureID,
                    'Overview': row.get('강의소개', ''),
                    'evaluationRatio': row.get('영어강의 비율', ''),
                    'isTBL': convert_to_yn(row.get('TBL강의', '')),
                    'isPBL': convert_to_yn(row.get('PBL강의', '')),
                    'isSeminar': convert_to_yn(row.get('세미나강의', '')),
                    'isSmall': convert_to_yn(row.get('소규모강의', '')),
                    'isConvergence': convert_to_yn(row.get('융합강의', '')),
                    'isTeamTeaching': convert_to_yn(row.get('팀티칭강의', '')),
                    'isFocus': convert_to_yn(row.get('집중이수제강의', '')),
                    'isArt': convert_to_yn(row.get('예체능', ''))
                }

                # SQL 쿼리 생성
                columns = ', '.join(insert_data.keys())
                placeholders = ', '.join(['?' for _ in insert_data])
                sql = f'''
                INSERT INTO LectureDetailData ({columns})
                VALUES ({placeholders})
                '''

                # 데이터 삽입 실행
                cursor.execute(sql, list(insert_data.values()))

# 변경사항 저장 및 연결 종료
conn.commit()
conn.close()

print("데이터 가져오기가 완료되었습니다.")

In [6]:
import csv
import sqlite3

# 데이터베이스 연결
try:
    conn = sqlite3.connect('kwu-lecture-recommendation-v5.db')
    cursor = conn.cursor()
    print("데이터베이스 연결 성공")
except sqlite3.Error as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit(1)

# CSV 파일 읽기
try:
    with open('수강인원즈.csv', 'r', encoding='cp949') as file:
        csv_reader = csv.DictReader(file)
        print("CSV 파일 열기 성공")
        
        # 현재 년도와 학기 설정
        current_year = 24
        current_semester = '2학기'  # 또는 '2학기'에 따라 조정
        print(f"현재 년도: {current_year}, 학기: {current_semester}")

        row_count = 0
        update_count = 0
        
        for row in csv_reader:
            row_count += 1
            # 학정번호가 있는 경우에만 처리
            if row['학정번호']:
                print(f"\n처리 중인 학정번호: {row['학정번호']}")

                # LectureList 테이블에서 lectureID 조회
                cursor.execute('''
                SELECT lectureID FROM LectureList
                WHERE year = ? AND semester = ? AND lecNumber = ?
                ''', (current_year, current_semester, row['학정번호']))

                lecture_id_row = cursor.fetchone()

                if lecture_id_row:
                    lectureID = lecture_id_row[0]
                    print(f"찾은 lectureID: {lectureID}")

                    # 과거 3년간의 수강 인원 데이터 준비
                    taken_people = {
                        'takenPeople1yearsAgo': row.get('2023', 0),
                        'takenPeople2yearsAgo': row.get('2022', 0),
                        'takenPeople3yearsAgo': row.get('2021', 0)
                    }
                    print(f"수강 인원 데이터: {taken_people}")

                    # SQL 쿼리 생성 (UPDATE 사용)
                    sql = '''
                    UPDATE LectureDetailData
                    SET takenPeople1yearsAgo = ?,
                        takenPeople2yearsAgo = ?,
                        takenPeople3yearsAgo = ?
                    WHERE lectureID = ?
                    '''

                    # 데이터 업데이트 실행
                    try:
                        cursor.execute(sql, (
                            taken_people['takenPeople1yearsAgo'],
                            taken_people['takenPeople2yearsAgo'],
                            taken_people['takenPeople3yearsAgo'],
                            lectureID
                        ))
                        update_count += 1
                        print(f"업데이트 성공: lectureID {lectureID}")
                    except sqlite3.Error as e:
                        print(f"업데이트 중 오류 발생: {e}")
                else:
                    print(f"lectureID를 찾을 수 없음: {row['학정번호']}")
            else:
                print("학정번호가 비어있는 행 발견")

        print(f"\n총 처리된 행 수: {row_count}")
        print(f"성공적으로 업데이트된 행 수: {update_count}")

        # 변경사항 저장
        conn.commit()
        print("모든 변경사항이 저장되었습니다.")

except FileNotFoundError:
    print("CSV 파일을 찾을 수 없습니다.")
except csv.Error as e:
    print(f"CSV 파일 읽기 오류: {e}")
except sqlite3.Error as e:
    print(f"데이터베이스 오류: {e}")
finally:
    # 연결 종료
    if conn:
        conn.close()
        print("데이터베이스 연결이 종료되었습니다.")

print("프로그램 실행이 완료되었습니다.")

데이터베이스 연결 성공
CSV 파일 열기 성공
현재 년도: 24, 학기: 2학기

처리 중인 학정번호: 0000-1-8128-01
찾은 lectureID: 3113
수강 인원 데이터: {'takenPeople1yearsAgo': '1040', 'takenPeople2yearsAgo': '993', 'takenPeople3yearsAgo': '993'}
업데이트 성공: lectureID 3113

처리 중인 학정번호: 0000-1-8128-02
찾은 lectureID: 3114
수강 인원 데이터: {'takenPeople1yearsAgo': '125', 'takenPeople2yearsAgo': '97', 'takenPeople3yearsAgo': ''}
업데이트 성공: lectureID 3114

처리 중인 학정번호: I000-1-3362-01
찾은 lectureID: 3115
수강 인원 데이터: {'takenPeople1yearsAgo': '', 'takenPeople2yearsAgo': '', 'takenPeople3yearsAgo': ''}
업데이트 성공: lectureID 3115

처리 중인 학정번호: I000-1-3362-02
찾은 lectureID: 3116
수강 인원 데이터: {'takenPeople1yearsAgo': '15', 'takenPeople2yearsAgo': '14', 'takenPeople3yearsAgo': '29'}
업데이트 성공: lectureID 3116

처리 중인 학정번호: I000-1-3362-03
찾은 lectureID: 3117
수강 인원 데이터: {'takenPeople1yearsAgo': '30', 'takenPeople2yearsAgo': '30', 'takenPeople3yearsAgo': '7'}
업데이트 성공: lectureID 3117

처리 중인 학정번호: I000-1-3362-04
찾은 lectureID: 3118
수강 인원 데이터: {'takenPeople1yearsAgo': '30', 'take

In [2]:
import csv
import sqlite3

# 데이터베이스 연결
try:
    conn = sqlite3.connect('kwu-lecture-recommendation-v5.db')
    cursor = conn.cursor()
    print("데이터베이스 연결 성공")
except sqlite3.Error as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit(1)

# CSV 파일 읽기
try:
    with open('수강일정.csv', 'r', encoding='cp949') as file:
        csv_reader = csv.DictReader(file)
        print("CSV 파일 열기 성공")
        
        # 현재 년도와 학기 설정
        current_year = 24
        current_semester = '2학기'  # 또는 '2학기'에 따라 조정
        print(f"현재 년도: {current_year}, 학기: {current_semester}")

        row_count = 0
        update_count = 0
        
        for row in csv_reader:
            row_count += 1
            # 학정번호가 있는 경우에만 처리
            if row['학정번호']:
                print(f"\n처리 중인 학정번호: {row['학정번호']}")

                # LectureList 테이블에서 lectureID 조회
                cursor.execute('''
                SELECT lectureID FROM LectureList
                WHERE year = ? AND semester = ? AND lecNumber = ?
                ''', (current_year, current_semester, row['학정번호']))

                lecture_id_row = cursor.fetchone()

                if lecture_id_row:
                    lectureID = lecture_id_row[0]
                    print(f"찾은 lectureID: {lectureID}")

                    # 과거 3년간의 수강 인원 데이터 준비
                    taken_people = {
                        'scheduleNcontent': row.get('수강일정3', 0),
                    }
                    print(f"수강 인원 데이터: {taken_people}")

                    # SQL 쿼리 생성 (UPDATE 사용)
                    sql = '''
                    UPDATE LectureDetailData
                    SET scheduleNcontent = ?
                    WHERE lectureID = ?
                    '''

                    # 데이터 업데이트 실행
                    try:
                        cursor.execute(sql, (
                            taken_people['scheduleNcontent'],
                            lectureID
                        ))
                        update_count += 1
                        print(f"업데이트 성공: lectureID {lectureID}")
                    except sqlite3.Error as e:
                        print(f"업데이트 중 오류 발생: {e}")
                else:
                    print(f"lectureID를 찾을 수 없음: {row['학정번호']}")
            else:
                print("학정번호가 비어있는 행 발견")

        print(f"\n총 처리된 행 수: {row_count}")
        print(f"성공적으로 업데이트된 행 수: {update_count}")

        # 변경사항 저장
        conn.commit()
        print("모든 변경사항이 저장되었습니다.")

except FileNotFoundError:
    print("CSV 파일을 찾을 수 없습니다.")
except csv.Error as e:
    print(f"CSV 파일 읽기 오류: {e}")
except sqlite3.Error as e:
    print(f"데이터베이스 오류: {e}")
finally:
    # 연결 종료
    if conn:
        conn.close()
        print("데이터베이스 연결이 종료되었습니다.")

print("프로그램 실행이 완료되었습니다.")

데이터베이스 연결 성공
CSV 파일 열기 성공
현재 년도: 24, 학기: 2학기

처리 중인 학정번호: 0000-1-8128-01
찾은 lectureID: 3113
수강 인원 데이터: {'scheduleNcontent': '광운의 어제와 오늘||광운 동문들의 역사||알아두면 유익한 학교 기관 소개|광운역량이력서 작성|광운공감 교양교육 목표 및 균형이수체계|광운역량이력서 작성|직업 정보 탐색하기|CAT 검사 실시|나의 SWOT (강점, 약점, 기회, 위협) 분석|CAT 검사 실시|효과적인 자기소개서 작성법|CAT 검사 실시|리더쉽과 팔로우쉽|'}
업데이트 성공: lectureID 3113

처리 중인 학정번호: 0000-1-8128-02
찾은 lectureID: 3114
수강 인원 데이터: {'scheduleNcontent': '오리엔테이션|09/06 대면수업|학교 기관 및 학과소개, 학과별 교과목 이수체계 안내|09/13 대면수업|외국인 학부 유학생을 위한 대학생활 안내 1 (Klas 이용법)|09/20 대면수업|외국인 학부 유학생을 위한 대학생활 안내 2 (교수님께 이메일보내기, 외국인유학생 교과목소개, 휴대전화/SNS 예절)|09/27 대면수업|외국인 학부 유학생을 위한 대학생활 안내 3 (출입국 및 체류자격, 장학제도, 유학생특별프로그램 등)|10/04 대면수업 (변경될 수 있음)|국제교류팀, KIF, 유학생회 소개|10/11 대면수업|한국어 공부법|10/18 녹화동영상|대학생활과 시간관리|10/25 녹화동영상'}
업데이트 성공: lectureID 3114

처리 중인 학정번호: 3000-1-3362-01
찾은 lectureID: 3135
수강 인원 데이터: {'scheduleNcontent': '- Class Introduction - Unit 1: Reading 1: Small Talk: A Big Deal|대면수업|- Unit 1: Reading 2: 21st Century Job Interviews - Unit 1: Writing/Grammar|대